# Packet analysis of a TCP Network

Using the SUEE1 dataset, we will analyse TCP packet data of a 'eduroam' network and build three machine learning classifiers with the objective of identifying IPs running slowloris attack inside the network. 

In [1]:
# Lib importing
import os
import re

import pandas as pd





In [2]:
# Setting global vars
DATASETS_PATH = "datasets/"

In [3]:
# Reading the original, unchanged dataset after .CSV export
original_df = pd.read_csv(os.path.join(DATASETS_PATH, "original_dataset.csv"), sep=',', index_col="frame.number")

In [4]:
# Number of lines inside the dataset
print("Dataset number of lines: {}".format(len(original_df)))

Dataset number of lines: 2089436


In [5]:
# A little look at our dataset
original_df.head()

,frame.len,frame.protocols,eth.src,eth.dst,ip.proto,ip.len,ip.src,ip.dst,ip.ttl,tcp.flags.res,...,tcp.flags.push,tcp.flags.reset,tcp.flags.syn,tcp.flags.fin,tcp.stream,tcp.seq_raw,tcp.seq,tcp.window_size_value,tcp.time_relative,tcp.time_delta
frame.number,,,,,,,,,,,,,,,,,,,,,
1,66,eth:ethertype:ip:tcp,ff:ff:ff:ff:00:01,ff:ff:ff:ff:00:02,6,52,192.168.0.1,192.168.0.2,64,0,...,0,0,0,1,0,1202351615,1,235,0.000000,0.000000
2,66,eth:ethertype:ip:tcp,ff:ff:ff:ff:00:03,ff:ff:ff:ff:00:01,6,52,192.168.0.2,192.168.0.1,62,0,...,0,0,0,0,0,2705104314,1,245,0.038679,0.038679
3,54,eth:ethertype:ip:tcp,ff:ff:ff:ff:00:01,ff:ff:ff:ff:00:02,6,40,192.168.0.1,192.168.0.3,64,0,...,0,0,0,1,1,3098131932,1,237,0.000000,0.000000
4,54,eth:ethertype:ip:tcp,ff:ff:ff:ff:00:03,ff:ff:ff:ff:00:01,6,40,192.168.0.3,192.168.0.1,126,0,...,0,0,0,0,1,1558776769,1,254,0.001364,0.001364
5,66,eth:ethertype:ip:tcp,ff:ff:ff:ff:00:01,ff:ff:ff:ff:00:02,6,52,192.168.0.1,192.168.0.4,64,0,...,0,0,0,1,2,1199048573,1,235,0.000000,0.000000


In [6]:
# Columns and their respective type
original_df.dtypes

frame.len                  int64
frame.protocols           object
eth.src                   object
eth.dst                   object
ip.proto                   int64
ip.len                     int64
ip.src                    object
ip.dst                    object
ip.ttl                     int64
tcp.flags.res              int64
tcp.flags.ns               int64
tcp.flags.urg              int64
tcp.flags.ack              int64
tcp.flags.push             int64
tcp.flags.reset            int64
tcp.flags.syn              int64
tcp.flags.fin              int64
tcp.stream                 int64
tcp.seq_raw                int64
tcp.seq                    int64
tcp.window_size_value      int64
tcp.time_relative        float64
tcp.time_delta           float64
dtype: object

In [7]:
# Statistical analysis of the quantitative columns
original_df[['frame.len',
             'ip.len',
             'ip.ttl',
             'tcp.window_size_value',
             'tcp.time_relative',
             'tcp.time_delta' 
           ]].describe()

,frame.len,ip.len,ip.ttl,tcp.window_size_value,tcp.time_relative,tcp.time_delta
count,2.089436e+06,2.089436e+06,2.089436e+06,2.089436e+06,2.089436e+06,2.089436e+06
mean,6.234917e+01,7.901674e+02,6.643968e+01,1.011751e+04,-2.929557e+01,-5.852291e-01
std,9.250634e+00,2.815735e+03,1.859239e+01,1.862629e+04,3.708226e+02,6.531709e+01
min,5.400000e+01,4.000000e+01,3.700000e+01,0.000000e+00,-4.293922e+03,-4.293949e+03
25%,5.400000e+01,4.400000e+01,6.200000e+01,2.350000e+02,1.067800e-02,7.000000e-06
50%,6.600000e+01,5.200000e+01,6.400000e+01,3.050000e+02,2.354410e-01,4.300000e-05
75%,6.600000e+01,5.570000e+02,6.400000e+01,8.192000e+03,1.545133e+00,1.274000e-03
max,5.370000e+02,6.521200e+04,2.540000e+02,6.553500e+04,2.689156e+03,2.684000e+03


In [8]:
# Checking how many unique IP addresses there's on the dataset
print("IP sources: {}\n"\
       "IP destinations: {}".format(len(original_df['ip.src'].value_counts()), len(original_df['ip.dst'].value_counts())))

IP sources: 1859
IP destinations: 1858


## Data cleaning

In [9]:
# Discarding useless columns
new_df = original_df.drop(labels=[
    'frame.len',      # Not needed 
    'frame.protocols',# Not needed'
    'eth.src',        # Redundant w/ ip.src
    'eth.dst',        # Redundant w/ ip.dst
    'ip.proto',       # Always 6 (TCP)
    'ip.len',         # Not needed
    'ip.dst',         # Doesn't matter who's being attacked
    'tcp.seq_raw'     # Not needed
], axis=1)

In [10]:
# Grouping and generating aggregates
new_df = new_df.groupby(['tcp.stream', 'ip.src']).agg({
    'ip.ttl': ['mean', 'median'],
    'tcp.time_relative': ['max', 'min', 'mean', 'median'],
    'ip.ttl': ['mean', 'median', 'max', 'min'],
    'tcp.flags.res': ['sum'],
    'tcp.flags.ns': ['sum'],
    'tcp.flags.urg': ['sum'],
    'tcp.flags.ack': ['sum'],
    'tcp.flags.push': ['sum'],
    'tcp.flags.reset': ['sum'],
    'tcp.flags.syn': ['sum'],
    'tcp.flags.fin': ['sum'],
    'tcp.seq': ['max'],
    'tcp.window_size_value': ['max', 'min', 'mean', 'median'],
    'tcp.time_relative': ['max', 'min', 'mean', 'median'],
    'tcp.time_delta': ['max', 'min', 'mean', 'median']
})

# Flattening new columns to index 0
new_df.columns = ["_".join(a) for a in new_df.columns.to_flat_index()]

# Flattening MultiIdx
new_df.index = [f"{num}_{value[1]}" for num, value in enumerate(new_df.index.to_flat_index())]

In [11]:
# Generating labels
new_df['attacker'] = new_df.index.str.match('.+_10\.128\.0\.[012]?[0-9][0-9]').astype(int) # Match IP addresses from 10.128.0.0 to 10.128.0.256

In [12]:
# Saving cleansed dataset

new_df.to_csv(os.path.join(DATASETS_PATH, 'labeled_dataset.csv'), sep=',')
new_df.head()

,ip.ttl_mean,ip.ttl_median,ip.ttl_max,ip.ttl_min,tcp.time_relative_max,tcp.time_relative_min,tcp.time_relative_mean,tcp.time_relative_median,tcp.flags.res_sum,tcp.flags.ns_sum,...,tcp.seq_max,tcp.window_size_value_max,tcp.window_size_value_min,tcp.window_size_value_mean,tcp.window_size_value_median,tcp.time_delta_max,tcp.time_delta_min,tcp.time_delta_mean,tcp.time_delta_median,attacker
0_192.168.0.1,64.0,64.0,64,64,0.000000,-2792.830719,-1396.415360,-1396.415360,0,0,...,2,235,0,117.500000,117.5,0.000034,0.000000,0.000017,0.000017,0
1_192.168.0.2,62.0,62.0,62,62,0.038679,-2792.830753,-1396.396037,-1396.396037,0,0,...,1,245,245,245.000000,245.0,0.038679,-2792.869432,-1396.415377,-1396.415377,0
2_192.168.0.1,64.0,64.0,64,64,2.544811,0.000000,1.272406,1.272406,0,0,...,2,237,237,237.000000,237.0,0.000013,0.000000,0.000006,0.000006,0
3_192.168.0.3,126.0,126.0,126,126,2.544837,0.001364,1.697000,2.544798,0,0,...,2,254,0,169.333333,254.0,2.543434,0.000026,0.848275,0.001364,0
4_192.168.0.1,64.0,64.0,64,64,6.003576,0.000000,3.001788,3.001788,0,0,...,2,235,235,235.000000,235.0,0.000021,0.000000,0.000010,0.000010,0
